Prediction- Weighted sum

In [69]:
import import_ipynb
from Item_Similarity_Computation import Similarity
from datetime import datetime
import operator
import sklearn.metrics
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors as NN
from Item_Similarity_Computation import Similarity
from prediction_computation import prediction_compute
from sklearn.metrics import r2_score 

#data,X,y,pivot=dataframe.dataset()

class estimator(BaseEstimator,ClassifierMixin):
    #BaseEstimator is for gerparam and set params
    
    def __init__(self,metric='cosine',n_neighbors=3,pred='weighted_sum'):
        
        self.n_neighbors=n_neighbors
        self.metric=metric
        self.my_metric=Similarity
        self.pivot=None
        self.dataset=None
        self.similarity_matrix=None
        self.X=None
        self.y=None
        self.pred=pred
        self.my_predict=prediction_compute

    '''
    First, I am going to define knn using a metric in Item Similarity Computation.ipynb
    
    '''
    #each estimator needs a fit, predict and score function.
        
    def fit(self,X,y):
     
        self.X=X
        self.y=y
        self.dataset=self.X.copy()
        self.dataset['rating']=self.y
        self.pivot=self.dataset.pivot_table(index="movieId",columns="userId",values="rating").fillna(0)
        self.similarity_matrix=self.my_metric.sim(self.metric,self.pivot)
        return self
       
    '''in the following I am going to define three functions. predict is the function with two entries. one of them is the estimator
    (which stores the data about X_train after fitting. the other one is the new data we want to predict.
    '''
    
    def distance (self,F):
    
       
        '''
        when we have a data to predict, the similarities between items in this data set should be taken from the similartities we
        found  before (in training data). As the knn model is used in the paper so for the target item i we are just looking for
        similar items in its defined neighborhood
    
        '''
        
        for c in F.columns:
            
          #here we are jst taking acount those entries similar to the target i which are in the k-neighborhood of the item i.
            similarity_matrix_zero=F.copy()
            sort_column=similarity_matrix_zero.sort_values(by = [c],ascending=False).index
    
            index_n=sort_column[0:self.n_neighbors]
            
            similarity_matrix_zero.loc[list(set(similarity_matrix_zero.index.tolist()).difference (set(index_n.tolist()))),c]=0
            
            '''Now we have a matrix with non-xero entries juft for those pair in each other neighborhood'''

        return similarity_matrix_zero

        
    def predict(self,data_predict):
        
  
        
        '''
        To predict a dataset, we have to use one of the prediction_computation models like weighted sum. For that we need two
        datasets including pivot and similarity. As it is a test data and we do not have ratings here, we have to use self.pivot 
        and self.similarity.
        
        '''
        #data_predict_pivot: it should have users in data_predict and all movies
        #data_predict_similarity: it should have movies in data_predict and all movies(because we are looking for movies in 
        #training data  which are similar to movie i in data_predict)
        
        predicted_users=list(set(data_predict.loc[:,'userId'].values))
        all_movies=set((data_predict.loc[:,'movieId'].values)).union (self.pivot.index.values)
    
        predicted_movies=np.unique(data_predict['movieId'].values).tolist()
    
        data_predict_pivot=self.pivot.reindex(index=predicted_users,columns=all_movies)
        data_predict_similarity=self.similarity_matrix.reindex(index=all_movies,columns=predicted_movies)
        '''
        Now we have a similairt dataset where each of the predicted movies can be the target and 
        its similarity with any other movies is considered.
        
        '''
        
        for i,j in zip(predicted_movies, predicted_movies):
            '''
            here, we have to put zero each entry of the diameter. Az it is the similairty of an item with itselves and we do not 
            want to consider it in the neighborhood.
            '''
            data_predict_similarity.loc[i, j]=0
        
        data_predict_pivot.iloc[:,:] = np.nan_to_num(data_predict_pivot) 
        data_predict_similarity.iloc[:,:] = np.nan_to_num(data_predict_similarity)
        
        
        predictions=self.my_predict.pre_model(self.pred,data_predict_pivot,self.distance(data_predict_similarity))
        
        '''
        this is the predicted scores for each predicted user and predicted item in the dataset given to predict function.
        
        '''
        predictions = predictions.reindex(index=predicted_users, columns = predicted_movies)
       
        y_hat=[]

        for i in data_predict.itertuples(index=False):
            #i has two component. the first one is the userId and the second one is the movieId
            j = i[0]
            k = i[1]
            y_hat.append(predictions.loc[j,k])
        return y_hat



    def score(self,X,y):
        #in this function we want to calculate the MAE egarding y hat and the real target. So at first, we need to predict X.
        predictions = self.predict(X)
        predictions = np.nan_to_num(predictions) 
        MAE = sklearn.metrics.mean_absolute_error(y,predictions)
        R_square = r2_score(y,predictions) 
        return -MAE